маголего

In [22]:
import requests 
from bs4 import BeautifulSoup
import time 
import random
import pandas as pd

In [23]:
def get_page(html_doc): #функция для страниц 1-14
    with open(html_doc, 'r', encoding='utf-8') as fp:
        tree = BeautifulSoup(fp, 'html.parser')

    films = tree.find_all('div', {'class' : 'styles_root__ti07r'})

    infa = []
    
    for film in films:
        a = film.find('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}).text #вытастикиваем страну
        films_list = a.split()
        country = films_list[:2]
        country_final = []
        stupid_sym = ['•']
        for elem in country:
            if elem not in stupid_sym:
                country_final.append(elem)
        strana = ' '.join(country_final)
        
        #вытастикиваем жанр
        a = film.find('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}).text 
        a_list = a.split()
        genre_list = a_list[2:]
        genre_final = []
        stupid_sym = ['•']
        for elem in genre_list:
            if elem not in stupid_sym:
                genre_final.append(elem)
        genre = ''.join(genre_final[0])
        
        #вытастикиваем количество оценок
        a = list(film.find('span', {'class': 'styles_kinopoiskCount__2_VPQ'}).text.strip(' '))
        amm = [ ] 
        for elem in a:
            if elem !=' ':
                amm.append(elem)
        nuber_of_rates = ''.join(amm)
        n_o_r = int(nuber_of_rates)
        
        #вытастикиваем год производства фильма
        b = film.find('span', {'class': 'desktop-list-main-info_secondaryText__M_aus'}).text
        b_list = b.split()
        if len(b_list) == 3:
            year = int(b_list[0].strip(','))
        else:
            year = int(b_list[-3].strip(','))
            
        #вытаскиваем режиссера
        d = film.find('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}).text 
        d_list = d.split()
        directors_list = d_list[2:]
        directors_almost_final = [ ]
        stupid_sym = ['•']
        for elem in directors_list:
            if elem not in stupid_sym:
                directors_almost_final.append(elem)
        directors_final_list = directors_almost_final[2:]
        director = ' '.join(directors_final_list)
        
        #вытаскиваем актера, исполняющего главную роль
        d1 = film.get_text(' ')
        d_list_1 = d1.split('•')
        d1_cut = d_list_1[1]
        d_list_2 = d1_cut.split(':')
        if len(d_list_2) >= 3:
            d1_cut_more = d_list_2[2]
        else:
            continue
        d_list_3 = d1_cut_more.split(', ')
        if len(d_list_3) >= 2:
            actor_1 = d_list_3[0].strip()
        else:
            s = ' '.join(d_list_3)
            s_list = s.split()
            actor_1 = ' '.join(s_list[:2])
        
        #конечный шаг для формирования датафрейма: с перемнными название фильма и рейтинг тут все довольно просто,поэтому дополнительных шагов не требуется
        infa.append(
            {'title' : film.find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text,
             'rating' : float(film.find('span', {'class': 'styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg'}).text),
             'year' : year,
             'country': strana,
             'genre' : genre,
             'number of rates': n_o_r,
             'director' : director,
             'actor_1' : actor_1}
        )

    return infa

In [24]:
pd.DataFrame(get_page('page 1.html')).head() #проверяем работу функции на выгрузке данных из одной страницы

,title,rating,year,country,genre,number of rates,director,actor_1
0,Побег из Шоушенка,9.1,1994,США,драма,832942,Фрэнк Дарабонт,Тим Роббинс
1,Крестный отец,8.7,1972,США,драма,330352,Фрэнсис Форд Коппола,Марлон Брандо
2,Леон,8.7,1994,Франция,боевик,554085,Люк Бессон,Жан Рено
3,Бойцовский клуб,8.6,1999,США,триллер,668532,Дэвид Финчер,Эдвард Нортон
4,Назад в будущее,8.6,1985,США,фантастика,532432,Роберт Земекис,Майкл Дж. Фокс


In [25]:
from tqdm.notebook import tqdm

filenames = [
    'page 1.html',
    'page 2.html',
    'page 3.html',
    'page 4.html',
    'page 5.html',
    'page 6.html',
    'page 7.html',
    'page 8.html',
    'page 9.html',
    'page 10.html',
    'page 11.html',
    'page 12.html',
    'page 13.html',
    'page 14.html'
]
infa = []

for filename in tqdm(filenames):
    infa += get_page(filename)


df1 = pd.DataFrame(infa)

  0%|          | 0/14 [00:00<?, ?it/s]

In [26]:
df1 = pd.DataFrame(infa) 
print(df1.shape)
df1

(684, 8)


,title,rating,year,country,genre,number of rates,director,actor_1
0,Побег из Шоушенка,9.1,1994,США,драма,832942,Фрэнк Дарабонт,Тим Роббинс
1,Крестный отец,8.7,1972,США,драма,330352,Фрэнсис Форд Коппола,Марлон Брандо
2,Леон,8.7,1994,Франция,боевик,554085,Люк Бессон,Жан Рено
3,Бойцовский клуб,8.6,1999,США,триллер,668532,Дэвид Финчер,Эдвард Нортон
4,Назад в будущее,8.6,1985,США,фантастика,532432,Роберт Земекис,Майкл Дж. Фокс
...,...,...,...,...,...,...,...,...
679,Хэллоуин,7.0,1978,США,ужасы,37589,Джон Карпентер,Дональд Плезенс
680,Рискованный бизнес,7.0,1983,США,драма,7618,Пол Брикман,Том Круз
681,Игрок,7.0,1992,США,триллер,5001,Роберт Олтмен,Тим Роббинс
682,Винтовая лестница,7.0,1946,США,ужасы,891,Роберт Сьодмак,Дороти МакГуайр


In [27]:
df1.pivot_table(index='director', values='rating', aggfunc='count') #проверка отсутсвия отсутствия одинаковых наблюдений в выборке

,rating
director,
Аббас Киаростами,2
Абель Ганс,1
Абель Феррара,1
Айвен Райтман,1
Аки Каурисмяки,1
...,...
Эррол Моррис,1
Юэнь У-Пин,1
Ян Шванкмайер,1


In [28]:
def get_page_extra(html_doc): #функция для страницы 15
    with open(html_doc, 'r', encoding='utf-8') as fp:
        tree = BeautifulSoup(fp, 'html.parser')

    films = tree.find_all('div', {'class' : 'styles_root__ti07r'})

    infa_extra = []
    
    for film in films:
        a = film.find('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}).text #вытастикиваем страну
        films_list = a.split()
        country = films_list[:2]
        country_final = []
        stupid_sym = ['•']
        for elem in country:
            if elem not in stupid_sym:
                country_final.append(elem)
        strana = ' '.join(country_final)
        
        #вытастикиваем жанр
        a = film.find('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}).text 
        a_list = a.split()
        genre_list = a_list[2:]
        genre_final = []
        stupid_sym = ['•']
        for elem in genre_list:
            if elem not in stupid_sym:
                genre_final.append(elem)
        genre = ''.join(genre_final[0])
        
        #вытастикиваем количество оценок
        a = list(film.find('span', {'class': 'styles_kinopoiskCount__2_VPQ'}).text.strip(' '))
        amm = [ ] 
        for elem in a:
            if elem !=' ':
                amm.append(elem)
        nuber_of_rates = ''.join(amm)
        n_o_r = int(nuber_of_rates)
        
        #вытастикиваем год производства фильма
        b = film.find('span', {'class': 'desktop-list-main-info_secondaryText__M_aus'}).text
        b_list = b.split()
        if len(b_list) == 3:
            year = int(b_list[0].strip(','))
        else:
            year = int(b_list[-3].strip(','))
        
        #а тут все очень интересно: чтобы вытащить рейтинг для каждого фильма с этой страницы пришлось очень постараться...
        dict_films = { } 
        for i in range(0,11):
            a = films[i].find('span', {'class': 'styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg'}).text
            a1 = (films[i].find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text).strip()
            dict_films[a1] = a
        for i in range(11,50):
            b = films[i].find('span', {'class': 'styles_kinopoiskValueNeutral__sW9QT styles_kinopoiskValue__9qXjg'}).text
            b1 = (films[i].find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text).strip()
            dict_films[b1] = b
        film_title = (film.find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text).strip()
        rating = dict_films[film_title]
        
         #вытаскиваем режиссера
        d = film.find('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}).text 
        d_list = d.split()
        directors_list = d_list[2:]
        directors_almost_final = [ ]
        stupid_sym = ['•']
        for elem in directors_list:
            if elem not in stupid_sym:
                directors_almost_final.append(elem)
        directors_final_list = directors_almost_final[2:]
        director = ' '.join(directors_final_list)
        
        #вытаскиваем актера, исполняющего главную роль
        d1 = film.get_text(' ')
        d_list_1 = d1.split('•')
        d1_cut = d_list_1[1]
        d_list_2 = d1_cut.split(':')
        if len(d_list_2) >= 3:
            d1_cut_more = d_list_2[2]
        else:
            continue
        d_list_3 = d1_cut_more.split(', ')
        if len(d_list_3) >= 2:
            actor_1 = d_list_3[0].strip()
        else:
            s = ' '.join(d_list_3)
            s_list = s.split()
            actor_1 = ' '.join(s_list[:2])
         
        #конечный шаг для формирования датафрейма 
        infa_extra.append(
            {'title' : film.find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text,
             'rating' : float(rating),
             'year' : year,
             'country': strana,
             'genre' : genre,
             'number of rates': n_o_r,
             'director' : director,
             'actor_1' : actor_1}
        )

    return infa_extra

In [29]:
pd.DataFrame(get_page_extra('page 15.html')).head()

,title,rating,year,country,genre,number of rates,director,actor_1
0,Большие надежды,7.0,1946,Великобритания,драма,1496,Дэвид Лин,Джон Миллз
1,Оборотень,7.0,2000,Канада,ужасы,9579,Джон Фоусет,Эмили Перкинс
2,Приключения Робин Гуда,7.0,1938,США,боевик,1984,Майкл Кёртиц,Эррол Флинн
3,В упор,7.0,1967,США,триллер,1529,Джон Бурмен,Ли Марвин
4,Сладости жизни,7.0,1990,Великобритания,драма,1141,Майк Ли,Элисон Стэдмен


In [30]:
from tqdm.notebook import tqdm

filename = 'page 15.html'
infa_extra = []
infa_extra += get_page_extra(filename)


df2 = pd.DataFrame(infa_extra)

In [31]:
df2 = pd.DataFrame(infa_extra)
print(df2.shape)
df2.head()

(50, 8)


,title,rating,year,country,genre,number of rates,director,actor_1
0,Большие надежды,7.0,1946,Великобритания,драма,1496,Дэвид Лин,Джон Миллз
1,Оборотень,7.0,2000,Канада,ужасы,9579,Джон Фоусет,Эмили Перкинс
2,Приключения Робин Гуда,7.0,1938,США,боевик,1984,Майкл Кёртиц,Эррол Флинн
3,В упор,7.0,1967,США,триллер,1529,Джон Бурмен,Ли Марвин
4,Сладости жизни,7.0,1990,Великобритания,драма,1141,Майк Ли,Элисон Стэдмен


In [32]:
def get_page_extra_super(html_doc): #функция для страниц 16-19
    with open(html_doc, 'r', encoding='utf-8') as fp:
        tree = BeautifulSoup(fp, 'html.parser')

    films = tree.find_all('div', {'class' : 'styles_root__ti07r'})

    infa_extra_super = []
    
    for film in films:
        a = film.find('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}).text #вытастикиваем страну
        films_list = a.split()
        country = films_list[:2]
        country_final = []
        stupid_sym = ['•']
        for elem in country:
            if elem not in stupid_sym:
                country_final.append(elem)
        strana = ' '.join(country_final)
        
        #вытастикиваем жанр
        a = film.find('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}).text 
        a_list = a.split()
        genre_list = a_list[2:]
        genre_final = []
        stupid_sym = ['•']
        for elem in genre_list:
            if elem not in stupid_sym:
                genre_final.append(elem)
        genre = ''.join(genre_final[0])
        
        #вытастикиваем количество оценок
        a = list(film.find('span', {'class': 'styles_kinopoiskCount__2_VPQ'}).text.strip(' '))
        amm = [ ] 
        for elem in a:
            if elem !=' ':
                amm.append(elem)
        nuber_of_rates = ''.join(amm)
        n_o_r = int(nuber_of_rates)
        
        #вытастикиваем год производства фильма
        b = film.find('span', {'class': 'desktop-list-main-info_secondaryText__M_aus'}).text
        b_list = b.split()
        if len(b_list) == 3:
            year = int(b_list[0].strip(','))
        else:
            year = int(b_list[-3].strip(','))
            
        #вытаскиваем режиссера
        d = film.find('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}).text 
        d_list = d.split()
        directors_list = d_list[2:]
        directors_almost_final = [ ]
        stupid_sym = ['•']
        for elem in directors_list:
            if elem not in stupid_sym:
                directors_almost_final.append(elem)
        directors_final_list = directors_almost_final[2:]
        director = ' '.join(directors_final_list)
        
        #вытаскиваем актера, исполняющего главную роль
        d1 = film.get_text(' ')
        d_list_1 = d1.split('•')
        d1_cut = d_list_1[1]
        d_list_2 = d1_cut.split(':')
        if len(d_list_2) >= 3:
            d1_cut_more = d_list_2[2]
        else:
            continue
        d_list_3 = d1_cut_more.split(', ')
        if len(d_list_3) >= 2:
            actor_1 = d_list_3[0].strip()
        else:
            s = ' '.join(d_list_3)
            s_list = s.split()
            actor_1 = ' '.join(s_list[:2])
        
        #конечный шаг для формирования датафрейма 
        infa_extra_super.append(
            {'title' : film.find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text,
             'rating' : float(film.find('span', {'class': 'styles_kinopoiskValueNeutral__sW9QT styles_kinopoiskValue__9qXjg'}).text),
             'year' : year,
             'country': strana,
             'genre' : genre,
             'number of rates': n_o_r,
             'director' : director,
             'actor_1' : actor_1}
        )

    return infa_extra_super

In [33]:
from tqdm.notebook import tqdm

filenames = [
    'page 16.html',
    'page 17.html',
    'page 18.html',
    'page 19.html'
]

infa_extra_super = []

for filename in tqdm(filenames):
    infa_extra_super += get_page_extra_super(filename)


df3 = pd.DataFrame(infa_extra_super)

  0%|          | 0/4 [00:00<?, ?it/s]

In [34]:
df3 = pd.DataFrame(infa_extra_super)
print(df3.shape)
df3

(196, 8)


,title,rating,year,country,genre,number of rates,director,actor_1
0,Забегаловка,6.9,1982,США,мелодрама,2142,Барри Левинсон,Стив Гуттенберг
1,Местный герой,6.9,1983,Великобритания,драма,911,Билл Форсайт,Берт Ланкастер
2,Большой парад,6.9,1925,США,драма,764,Кинг Видор,Джон Гилберт
3,Большой ансамбль,6.9,1955,США,фильм-нуар,446,Джозеф Х. Льюис,Корнел Уайлд
4,Музей восковых фигур,6.9,1953,США,ужасы,2284,Андре Де Тот,Винсент Прайс
...,...,...,...,...,...,...,...,...
191,Так держать Клео!,6.1,1964,Великобритания,мелодрама,129,Джералд Томас,Сидни Джеймс
192,Суперфлай,6.1,1972,США,боевик,443,Гордон Паркс мл.,Рон О’Нил
193,Проклятые,6.1,1962,Великобритания,ужасы,240,Джозеф Лоузи,МакДональд Кэри
194,Большое путешествие,6.1,2004,Франция,драма,113,Исмаэль Феррухи,Николя Казале


In [35]:
def get_page_extra_super_mega(html_doc): #функция для страницы 20
    with open(html_doc, 'r', encoding='utf-8') as fp:
        tree = BeautifulSoup(fp, 'html.parser')

    films = tree.find_all('div', {'class' : 'styles_root__ti07r'})

    infa_extra_super_mega = []
    
    for film in films:
        a = film.find('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}).text #вытастикиваем страну
        films_list = a.split()
        country = films_list[:2]
        country_final = []
        stupid_sym = ['•']
        for elem in country:
            if elem not in stupid_sym:
                country_final.append(elem)
        strana = ' '.join(country_final)
        
        #вытастикиваем жанр
        a = film.find('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}).text 
        a_list = a.split()
        genre_list = a_list[2:]
        genre_final = []
        stupid_sym = ['•']
        for elem in genre_list:
            if elem not in stupid_sym:
                genre_final.append(elem)
        genre = ''.join(genre_final[0])
        
        
        #вытастикиваем число оценок
        dict_films_2 = { } 
        for i in range(0,42):
            a = films[i].find('span', {'class': 'styles_kinopoiskCount__2_VPQ'}).text
            amm = [ ] 
            for elem in a:
                if elem !=' ':
                    amm.append(elem)
            nuber_of_rates = ''.join(amm)
            n_o_r_almost = int(nuber_of_rates)
            a1 = (films[i].find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text).strip()
            dict_films_2[a1] = n_o_r_almost    
        for i in range(42,50):
            b = 0
            b1 = (films[i].find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text).strip()
            dict_films_2[b1] = b
        
        film_title = (film.find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text).strip()
        n_o_r = dict_films_2[film_title]
        
        #вытастикиваем год
        b = film.find('span', {'class': 'desktop-list-main-info_secondaryText__M_aus'}).text
        b_list = b.split()
        if len(b_list) == 3:
            year = int(b_list[0].strip(','))
        else:
            year = int(b_list[-3].strip(','))
        
        #вытастикиваем рейтинг
        dict_films = { } 
        for i in range(0,42):
            a = films[i].find('span', {'class': 'styles_kinopoiskValueNeutral__sW9QT styles_kinopoiskValue__9qXjg'}).text
            a1 = (films[i].find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text).strip()
            dict_films[a1] = a
        for i in range(42,50):
            b = 0
            b1 = (films[i].find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text).strip()
            dict_films[b1] = b
        film_title = (film.find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text).strip() 
        rating = dict_films[film_title]
        
        #вытаскиваем режиссера
        d = film.find('span', {'class': 'desktop-list-main-info_truncatedText__IMQRP'}).text 
        d_list = d.split()
        directors_list = d_list[2:]
        directors_almost_final = [ ]
        stupid_sym = ['•']
        for elem in directors_list:
            if elem not in stupid_sym:
                directors_almost_final.append(elem)
        directors_final_list = directors_almost_final[2:]
        director = ' '.join(directors_final_list)
        
        #вытаскиваем актера, исполняющего главную роль
        d1 = film.get_text(' ')
        d_list_1 = d1.split('•')
        d1_cut = d_list_1[1]
        d_list_2 = d1_cut.split(':')
        if len(d_list_2) >= 3:
            d1_cut_more = d_list_2[2]
        else:
            continue
        d_list_3 = d1_cut_more.split(', ')
        if len(d_list_3) >= 2:
            actor_1 = d_list_3[0].strip()
        else:
            s = ' '.join(d_list_3)
            s_list = s.split()
            actor_1 = ' '.join(s_list[:2])
        
        
        #конечный шаг для формирования датафрейма 
        infa_extra_super_mega.append(
            {'title' : film.find('span', {'class': 'styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj'}).text,
             'rating' : float(rating),
             'year' : year,
             'country': strana,
             'genre' : genre,
             'number of rates': n_o_r,
             'director' : director,
             'actor_1' : actor_1}
        )

    return infa_extra_super_mega

In [36]:
print(pd.DataFrame(get_page_extra_super_mega('page 20.html')).head()) #проверяем работу функции
len(pd.DataFrame(get_page_extra_super_mega('page 20.html')))

                     title  rating  year         country    genre  \
0             Дикие ангелы     6.0  1966             США   боевик   
1                   Болото     6.0  2001       Аргентина    драма   
2             В руках бога     6.0  2004             США  триллер   
3  Земля, забытая временем     6.0  1974  Великобритания  фэнтези   
4              Синяя лампа     6.0  1950  Великобритания  триллер   

   number of rates          director         actor_1  
0              538     Роджер Корман     Питер Фонда  
1              450  Лукресия Мартель  Мерседес Моран  
2              507  Лодж Х. Керриган    Дэмиэн Льюис  
3              355      Кевин Коннор     Даг МакКлёр  
4              185      Бэзил Дирден     Джек Уорнер  


48

In [37]:
from tqdm.notebook import tqdm

filename = 'page 20.html'
infa_extra_super_mega = []
infa_extra_super_mega += get_page_extra_super_mega(filename)


df4 = pd.DataFrame(infa_extra_super_mega)

In [38]:
df4 = pd.DataFrame(infa_extra_super_mega)
print(df4.shape)
df4

(48, 8)


,title,rating,year,country,genre,number of rates,director,actor_1
0,Дикие ангелы,6.0,1966,США,боевик,538,Роджер Корман,Питер Фонда
1,Болото,6.0,2001,Аргентина,драма,450,Лукресия Мартель,Мерседес Моран
2,В руках бога,6.0,2004,США,триллер,507,Лодж Х. Керриган,Дэмиэн Льюис
3,"Земля, забытая временем",6.0,1974,Великобритания,фэнтези,355,Кевин Коннор,Даг МакКлёр
4,Синяя лампа,6.0,1950,Великобритания,триллер,185,Бэзил Дирден,Джек Уорнер
5,Образцовый самец,6.0,2001,Германия,комедия,19910,Бен Стиллер,Бен Стиллер
6,Страховой агент,6.0,1991,Канада,драма,319,Атом Эгоян,Элиас Котеас
7,Морверн Каллар,6.0,2002,Великобритания,драма,922,Линн Рэмси,Саманта Мортон
8,Роллербол,6.0,1975,Великобритания,фантастика,863,Норман Джуисон,Джеймс Каан
9,Блюз членососа,6.0,1972,США,документальный,126,Роберт Фрэнк,Крис Коллинз


##### а теперь соединяем все полученные нами датафреймы в один, чтобы получить датафрейм размерностью (1000,6)

In [39]:
df = pd.concat([df1, df2, df3, df4]).reset_index(drop=True)
print(df.shape)
df

(978, 8)


,title,rating,year,country,genre,number of rates,director,actor_1
0,Побег из Шоушенка,9.1,1994,США,драма,832942,Фрэнк Дарабонт,Тим Роббинс
1,Крестный отец,8.7,1972,США,драма,330352,Фрэнсис Форд Коппола,Марлон Брандо
2,Леон,8.7,1994,Франция,боевик,554085,Люк Бессон,Жан Рено
3,Бойцовский клуб,8.6,1999,США,триллер,668532,Дэвид Финчер,Эдвард Нортон
4,Назад в будущее,8.6,1985,США,фантастика,532432,Роберт Земекис,Майкл Дж. Фокс
...,...,...,...,...,...,...,...,...
973,Товарищи,0.0,1986,Великобритания,драма,0,Билл Дуглас,Кит Аллен
974,"Рита, Сью и Боб тоже",0.0,1987,Великобритания,драма,0,Алан Кларк,Мишель Холмс
975,Птанг. Янг. Киппербанг.,0.0,1982,Великобритания,комедия,0,Майкл Аптед,Джон Альбазини
976,Сэр Генри с Равилсон Энд,0.0,1980,Великобритания,комедия,0,Стив Робертс,Тревор Ховард


In [40]:
df.to_csv('magolego_2.csv')